In [13]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

from matplotlib import pyplot as plot

In [14]:
df = pd.read_csv("./data/datasets/historical.data")
df = df.set_index("date").sort_index(ascending=True)
#display(df)
df_inside = df.loc[df["label"] == "inside"].drop(["label"], axis=1)
df_outside = df.loc[df["label"] == "outside"].drop(["label"], axis=1)
print("inside")
display(df_inside)
#print("outside")
#display(df_outside)

inside


,temperature
date,
2018-02-22 00:30:00,22.875
2018-02-22 01:00:00,22.937
2018-02-22 01:30:00,23.062
2018-02-22 02:00:00,23.125
2018-02-22 02:30:00,23.187
...,...
2020-11-30 21:30:00,24.812
2020-11-30 22:00:00,24.750
2020-11-30 22:30:00,24.500


In [20]:
inside_arr=np.array(df_inside)
outside_arr=np.array(df_outside)

#print(inside_arr)

#train_test_split(inside_arr, shuffle=False)

inside_total_len,_=inside_arr.shape

train_p = 0.70
test_p = 0.15
ivs_p = 0.15

train_len = round(inside_total_len*train_p)
test_len = round(inside_total_len*test_p)
ivs_len = round(inside_total_len*ivs_p)

model_len = train_len+test_len

total_used_len = model_len+ivs_len

inside_train = inside_arr[:train_len]
inside_test = inside_arr[train_len:model_len]
inside_ivs = inside_arr[model_len:total_used_len]

print("train")
print(inside_train)
print("test")
print(inside_test)
print("ivs")
print(inside_ivs)

train
[[22.875]
 [22.937]
 [23.062]
 ...
 [23.437]
 [23.562]
 [23.562]]
test
[[23.375]
 [23.437]
 [23.437]
 ...
 [23.687]
 [24.   ]
 [23.812]]
ivs
[[24.125]
 [24.   ]
 [24.   ]
 ...
 [24.5  ]
 [24.437]
 [24.562]]


In [22]:
# 2h30

per_min = (2*60) + 30
per_30_mins = per_min / 30

print("Number of Wanted Predictions: " + str(per_30_mins))

Number of Wanted Predictions: 5.0


One-step-ahead (auto-regressive)
<br>
One-step-ahead (auto-regressive with exogenous inputs)
<br>
Recursive
<br>
MIMO
<br>
Direct
<br>
Hybrid (Recursive and Direct)
<br>
Hybrid (Direct and MIMO)
<br>
<br>
MIMO is good too, but you would need to comput each steop individually to have a full array of predictions, as the other can execute all predictions concurrently
<br>
Direct and Hybrid(Direct and MIMO) are on top - they can execute all
<br>
<br>
Hybrid(Direct and MIMO) = [2,2,1] predictive models
<br>
MIMO = [1,1,1,1,1] predictive models